In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Colab 사용자용

In [2]:
from google.colab import drive
drive.mount('/gdrive')
# repo에 있는 exp 폴더를 불러오자.
# /content 에 파일을 두면 colab 에서 인식된다.
# 원본 파일 경로는 자신의 드라이브에 맞게 설정
!cp -r /gdrive/"My Drive"/develop/course-v3-master/course-v3-master/nbs/dl2/exp /content

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


## The forward and backward passes

In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
x_train,y_train,x_valid,y_valid = get_data() # MNIST 데이터를 가져온다. (이전 강의에서 다룸)

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std # 평균과 분산을 확인해본다.

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std) # 데이터를 정규화하자. (가우시안)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std # 정규분포를 따른다.

(tensor(0.0001), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c # 학습셋 개수, 인풋차원, 클래스 개수

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [0]:
# 히든 레이어의 뉴럴 수
nh = 50

In [0]:
# kaiming init / he init 을 단순화 하였다.
w1 = torch.randn(m,nh)/math.sqrt(m)     # 1st 레이어 (784, 50)
b1 = torch.zeros(nh)

w2 = torch.randn(nh,1)/math.sqrt(nh)    # 2nd 레이어 (50, 1) 
b2 = torch.zeros(1)                     # 여기서 아웃풋이 10 이 아니라 1인것은 단순화하기 위해서다. 추후에 변경함

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
def lin(x, w, b): return x@w + b    # 간단하게 구현한 Linear Layer 계산

In [0]:
t = lin(x_valid, w1, b1)

In [16]:
# kaiming init을 사용했기에 다음과 같은 평균과 표준편차를 구할 수 있었다.
t.mean(),t.std() # 거의 정규 분포를 따르는 값

(tensor(-0.0278), tensor(0.9444))

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
t = relu(lin(x_valid, w1, b1))

In [19]:
t.mean(),t.std()

(tensor(0.3603), tensor(0.5420))

relu를 지나고 나니 위에서 처럼 (mean, std) = (0, 1) 이 아니다.

왜그럴까? 

relu를 지나면서 0보다 작은 수는 모두 0으로 올라왔기 때문이다.
(8강 1:31:40 참고)

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [0]:
# kaiming init / he init for relu
# relu를 위해 초기화 방법을 위의 식으로 바꿔보자
# w1 = torch.randn(m,nh)/math.sqrt(m)   # previous
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [21]:
w1.mean(),w1.std()  # 표준편차가 1이 아닌 0.05로 작아짐

(tensor(0.0003), tensor(0.0505))

In [22]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()    # 대신 relu를 지나고 나서는 표준편차가 1에 가까워짐

(tensor(0.5851), tensor(0.8287))

### torch.nn 의 init 모듈을 사용해보자

In [0]:
#export
from torch.nn import init

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')    # init 모듈 내부에 구현되어있는 kaiming init
t = relu(lin(x_valid, w1, b1))

In [0]:
'''
mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_out'`` preserves the magnitudes in the
            backwards pass.

fan_in 과 fan_out 의 차이는 가중치의 분산을 forward 때 저장하는지, backward 때 저장하는지 차이다.
'''
init.kaiming_normal_??

In [26]:
w1.mean(),w1.std()

(tensor(-0.0001), tensor(0.0508))

In [27]:
t.mean(),t.std()    # 위에 수식으로 구현한 결과와 거의같다.

(tensor(0.5323), tensor(0.8023))

In [0]:
import torch.nn

In [29]:
print(w1.shape)
print(torch.nn.Linear(m,nh).weight.shape)

torch.Size([784, 50])
torch.Size([50, 784])


직접 Tensor로 구현한 w1의 shape은 [784, 50] 인데, nn.Linear(m.nh)로 구현하면 [50, 784] 이다. 

(m = 784, nh = 50)

왜 서로 반대의 shape을 가질까? 소스코드를 확인해보자.

In [0]:
"""
@weak_script_method
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
"""
torch.nn.Linear.forward??

forward 함수는 반환값으로 F.linear 즉, torch.nn.functional.linear 을 반환한다.

In [0]:
'''
@weak_script
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret
'''
torch.nn.functional.linear??

소스코드를 잘 보면 `ret = torch.addmm(bias, input, weight.t())` 부분에서 `weight.t()` 를 통해 Transpose 하는 부분이 있다. 

Transpose를 해서 계산을 하기 때문에 `torch.nn.Linear(m,nh).weight.shape` 이 m, nh 순서가 바뀐 [50, 784] 가 된다.

### Convolution 레이어는 어떻게 초기화될까?

In [0]:
'''
class Conv2d(_ConvNd): 

이 클래스는 _ConvNd 클래스를 상속받았다.
File:           /usr/local/lib/python3.6/dist-packages/torch/nn/modules/conv.py
을 참조해서 _ConvNd 클래스 소스코드를 열어보자.
'''
torch.nn.Conv2d??

In [0]:
'''
def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
'''
torch.nn.modules.conv._ConvNd.reset_parameters??

Conv 또한 `init.kaiming_uniform_(self.weight, a=math.sqrt(5))` 으로 초기화 됨을 알 수 있다.

### 모델을 다시 만들자

In [0]:
# 만약에 relu를 이렇게 정의한다면...?
def relu(x): return x.clamp_min(0.) - 0.5

In [35]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()  # 표준편차가 1에 더 가까워지는 효과를 얻을 수 있었다.
                    # 강의에서 경험상 relu 에 0.5 를 빼주는게 도움이 되었다고 알려줌

(tensor(0.0812), tensor(0.8520))

In [0]:
# 2개의 hidden layer를 지나는 모델
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [37]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 20.9 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

손실 함수를 정의해보자. cross-entropy를 사용하지 않고, 단순화 하기 위해 일단 MSE를 사용한다. (추후에 변경)

In [39]:
model(x_valid).shape

torch.Size([10000, 1])

모델이 반환하는 텐서의 크기가 torch.Size([10000, 1]) 이다.

이전 강의에서 다룬 `squeeze()` 메소드를 사용해서 (,1) 형태의 shape을 제거해야 `mse`를 사용할 수 있다. 

In [40]:
model(x_valid).squeeze(-1).shape    # (,1) 이 제거됨

torch.Size([10000])

In [0]:
#export
def mse(output, targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [44]:
preds.shape # 모델이 예측한 값. (아직 학습은 이뤄지지 않음)

torch.Size([50000, 1])

In [45]:
mse(preds, y_train) # MSE 가 매우 크다. 
                    # 이제 학습을 통해 MSE를 최소화 하자.

tensor(26.5701)

### Gradients and backward pass

기울기 계산과 역전파 수행

각각의 수식마다의 기울기 계산식을 만들자. (행렬 계산방법은 `The Matrix Calculus You Need For Deep Learning` 을 참고하자)

In [0]:
def mse_grad(inp, targ): 
    # 이전 레이어의 출력에 대한 손실의 기울기
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [0]:
def relu_grad(inp, out):
    # 입력 활성화함수에 대한 relu의 기울기
    inp.g = (inp>0).float() * out.g # relu grad: 0 or 1

In [0]:
def lin_grad(inp, out, w, b):
    # 입력에 대한 matmul의 기울기
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # backward 에서는 사실 loss 가 필요없다. loss를 구할 필요는 없긴함(수렴하는지 확인용)
    loss = mse(out, targ)
    
    # backward pass:
    # mse -> lin2 -> relu -> lin1 순서로 거꾸로 계산 (체인룰)
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

In [54]:
w1.g, b1.g # 이런식으로 .g 에 기울기가 저장된다.

(tensor([[ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635],
         [ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635],
         [ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635],
         ...,
         [ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635],
         [ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635],
         [ 0.0052,  0.3086,  0.3542,  ...,  0.0121, -0.2450,  0.6635]]),
 tensor([-0.0123, -0.7272, -0.8345,  0.0920,  0.6048,  0.1824, -0.2442,  0.0336,
         -0.3668,  0.5408, -0.8034,  0.2622, -0.3996,  0.3065,  0.9204,  1.5724,
         -0.0693,  1.0817,  0.1523,  0.2087, -0.4128, -0.2323,  1.0893, -1.2743,
         -0.0520, -0.3629, -0.0363,  0.1273,  0.3370,  0.2268,  0.7889,  0.3509,
          0.0781, -0.2372,  0.1027,  0.4583,  0.3019,  0.0180, -0.8386,  1.9626,
          0.3507, -0.3786, -0.3715, -0.3063,  1.0787,  0.2786, -0.0229, -0.0285,
          0.5773, -1.5632]))

In [0]:
# 나중에 테스트해보기 위해서 기울기를 다 저장해두자.
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

이제 파이토치의 autograd 를 사용해서 결과를 확인해보자.

autograd는 결국 위에서 작성한 `forward_and_backward` 를 수행하면서 생기는 기울기를 모두 추적하는 것이다.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

수식으로 구한 기울기와 autograd 가 구해준 기울기가 같음을 증명했다.

## Refactor model

### Layers as classes

레이어들을 각각 클래스로 만들자. 

`__call__` 은 forward, backward 는 backward 를 구현

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    # 위에서 정의했던 수식과 똑같다.
    def backward(self): 
        self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): 
        self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out

    # 위에서 정의했던 수식과 똑같다.
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
        
    # 위에서 정의했던 수식과 똑같다.
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]  # 사용할 Layer를 모두 넣어둠
        self.loss = Mse()

    # forward    
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    # backward
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4  # 기울기를 모두 None 으로 초기화
model = Model(w1, b1, w2, b2)

In [83]:
%time loss = model(x_train, y_train)

CPU times: user 115 ms, sys: 2.9 ms, total: 118 ms
Wall time: 119 ms


In [84]:
%time model.backward() # 시간이 많이 소요된다.

CPU times: user 3.62 s, sys: 3.64 s, total: 7.26 s
Wall time: 7.3 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

기존과 똑같은 결과를 반환함을 확인

### Module.forward()

`__call__` 을 사용하지 말고, forward 를 만들어 사용하자.

Module 클래스를 정의하고, 다른 Layer 들이 상속하게 한다.

In [0]:
class Module():
    # __call__ 이 호출되면 self.forward 수행
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        # self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0) # previous
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)    # 속도 향상을 위해 einsum 으로 바꿈
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [93]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 378 µs, total: 120 ms
Wall time: 120 ms


In [94]:
%time model.backward()  # 기존 보다 휠씬 빨라짐 (einsum)

CPU times: user 227 ms, sys: 11.4 ms, total: 238 ms
Wall time: 254 ms


In [0]:
# 검증
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

einsum 말고 Transpose를 이용한 구현

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        # self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)  # previous
        # self.w.g = torch.einsum("bi,bj->ij", inp, out.g)                      # previous
        self.w.g = inp.t() @ out.g                                              # transpose 해서 계산
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [98]:
%time loss = model(x_train, y_train)

CPU times: user 112 ms, sys: 2.86 ms, total: 115 ms
Wall time: 114 ms


In [99]:
%time model.backward()  # einsum을 사용하지 않아도 똑같이 빠르다.

CPU times: user 224 ms, sys: 9.96 ms, total: 234 ms
Wall time: 235 ms


In [0]:
# 검증
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

파이토치에 구현된 Linear, Relu, Module 클래스를 사용하자.

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def forward(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [109]:
%time loss = model(x_train, y_train)

CPU times: user 99.3 ms, sys: 5.09 ms, total: 104 ms
Wall time: 104 ms


In [110]:
%time loss.backward()   # 역시 파이토치에서 구현해둔 방법이 가장 빠르다.

CPU times: user 78.2 ms, sys: 4.1 ms, total: 82.3 ms
Wall time: 82.3 ms


## Export

In [106]:
!./notebook2script.py 02_fully_connected.ipynb

/bin/bash: ./notebook2script.py: No such file or directory
